In [2]:
import warnings

warnings.filterwarnings("ignore")

import pyspark.sql
from pyspark.sql.functions import *

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from wrangle import wrangle_311

spark = pyspark.sql.SparkSession.builder.getOrCreate()

df = wrangle_311(spark)
print("\ndf shape: (%d, %d)\n" % (df.count(), len(df.columns)))
df.show(1, vertical=True)

[wrangle.py] reading case.csv
[wrangle.py] handling data types
[wrangle.py] parsing dates
[wrangle.py] adding features
[wrangle.py] joining departments

df shape: (841704, 20)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                

> How many different cases are there, by department?

In [4]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'case_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district',
 'num_weeks_late',
 'zipcode',
 'case_age',
 'days_to_closed',
 'case_lifetime',
 'department',
 'dept_subject_to_SLA']

In [6]:
df.groupby("department").count().show()

+--------------------+------+
|          department| count|
+--------------------+------+
|         Solid Waste|279270|
|Animal Care Services|116915|
|Trans & Cap Impro...| 96193|
|  Parks & Recreation| 19907|
|    Customer Service|  2849|
|        Metro Health|  5163|
|        City Council|    33|
|DSD/Code Enforcement|321374|
+--------------------+------+



> Does the percentage of cases that are late vary by department?

In [35]:
nrows = df.count()
# groups by department and case_late
# takes a count of cases per department
# creates new column "percentage" divides count by nrows
# displays findings
# "percentage" is % of cases each department handles out of all cases

(
    df.groupby("department", "case_late")
    .count()
    .filter(df.case_late == 'true')
    .withColumn('percentage', (round((col('count')/ nrows), 3)))
    .show() 
)

# City Council had no late cases

+--------------------+---------+-----+----------+
|          department|case_late|count|percentage|
+--------------------+---------+-----+----------+
|    Customer Service|     true| 2010|     0.002|
|Trans & Cap Impro...|     true| 5411|     0.006|
|        Metro Health|     true|  829|     0.001|
|Animal Care Services|     true|23276|     0.028|
|  Parks & Recreation|     true| 3797|     0.005|
|         Solid Waste|     true|32945|     0.039|
|DSD/Code Enforcement|     true|26235|     0.031|
+--------------------+---------+-----+----------+



They do not vary by a lot. 

> On average, how late are the late cases by department?

In [38]:
#(
#    df.groupby("department")
#    .agg(mean("num_days_late"))
#    .show() 
#)

(
    df.groupby("department", "case_late")
    .agg(mean("num_days_late"))
    .filter(df.case_late == 'true')
    .show() 
)

+--------------------+---------+------------------+
|          department|case_late|avg(num_days_late)|
+--------------------+---------+------------------+
|    Customer Service|     true| 87.68385942150394|
|Trans & Cap Impro...|     true|10.603064680316946|
|        Metro Health|     true|6.5438133155476494|
|Animal Care Services|     true|23.458633245820124|
|  Parks & Recreation|     true|22.348910457867518|
|         Solid Waste|     true| 7.186821906120899|
|DSD/Code Enforcement|     true| 49.38428705358908|
+--------------------+---------+------------------+



> What is the service type that is the most late? Just for Parks & Rec?

In [49]:
sample = df.sample(fraction=0.01, seed=13).toPandas()
print("shape:", sample.shape)
sample.head(2)

shape: (8522, 20)


,case_id,case_opened_date,case_closed_date,case_due_date,case_late,num_days_late,case_closed,service_request_type,SLA_days,case_status,source_id,request_address,council_district,num_weeks_late,zipcode,case_age,days_to_closed,case_lifetime,department,dept_subject_to_SLA
0,1014127339,2018-01-01 06:58:00,2018-01-02 15:32:00,2018-01-17 08:30:00,False,-14.706620,True,Front Or Side Yard Parking,16.063796,Closed,svcCRMSS,"10133 FIGARO CANYON, San Antonio, 78251",004,-2.100946,78251,219,1.0,1,DSD/Code Enforcement,True
1,1014127368,2018-01-01 07:18:00,2018-01-02 15:22:00,2018-01-17 08:30:00,False,-14.713275,True,Right Of Way/Sidewalk Obstruction,16.049769,Closed,svcCRMSS,"10133 FIGARO CANYON, San Antonio, 78251",004,-2.101896,78251,219,1.0,1,DSD/Code Enforcement,True


In [59]:
(
    df.groupby("service_request_type", "case_late")
    .agg(mean("num_days_late"))
    .filter(df.case_late == 'true')
    .sort(desc('avg(num_days_late)'))
    .show(10) 
)
# top 10

+--------------------+---------+------------------+
|service_request_type|case_late|avg(num_days_late)|
+--------------------+---------+------------------+
|Zoning: Recycle Yard|     true|210.89201994318182|
|  Zoning: Junk Yards|     true|200.20517608494276|
|Structure/Housing...|     true|190.20707698509807|
|Donation Containe...|     true|171.09115313942615|
|Storage of Used M...|     true|163.96812829714287|
|Labeling for Used...|     true|162.43032902285717|
|Record Keeping of...|     true|153.99724039428568|
|Signage Requied f...|     true|151.63868055333333|
|Traffic Signal Gr...|     true|137.64583330000002|
|License Requied U...|     true|128.79828704142858|
+--------------------+---------+------------------+
only showing top 10 rows



In [61]:
(
    df
    .where(df["department"] == 'Parks & Recreation')
    .groupby("service_request_type", "case_late")
    .agg(mean("num_days_late"))
    .filter(df.case_late == 'true')
    .sort(desc('avg(num_days_late)'))
    .show(10) 
)

+--------------------+---------+------------------+
|service_request_type|case_late|avg(num_days_late)|
+--------------------+---------+------------------+
|Amenity Park Impr...|     true|    76.87018194568|
|Major Park Improv...|     true| 75.79450367282354|
|Reservation Assis...|     true|       66.03116319|
|Park Building Mai...|     true|59.367469802695666|
| Sportfield Lighting|     true|51.480049793645534|
|          Electrical|     true|42.951971658345116|
|        Tree Removal|     true| 40.28404224560715|
|Landscape Mainten...|     true| 38.86797941326316|
|     Heavy Equipment|     true| 38.56625730831579|
|Miscellaneous Par...|     true| 33.62472491253488|
+--------------------+---------+------------------+
only showing top 10 rows



> For the DSD/Code Enforcement department, what are the most common service request types? Look at other departments too.

In [66]:
(
    df
    .where(df.department == "DSD/Code Enforcement")
    .groupby("service_request_type")
    .count()
    .sort(desc("count"))
    .show(10)
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|Overgrown Yard/Trash|65895|
|        Bandit Signs|32910|
|Front Or Side Yar...|28794|
|Junk Vehicle On P...|21473|
|Alley-Way Mainten...|20214|
|Right Of Way/Side...|17699|
|Vacant Lot/Overgr...| 9329|
|Minimum Housing-O...| 8543|
|Graffiti: Private...| 8525|
|Dang. Premises/CL...| 8004|
+--------------------+-----+
only showing top 10 rows



In [90]:
d = df.select("department").distinct().toPandas()

In [89]:
for c in d.department:
    print(f"Department: {c}")
    (
    df
    .where(df.department == c)
    .groupby("service_request_type")
    .count()
    .sort(desc("count"))
    .show(1)
    )

Department: Solid Waste
+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|86855|
+--------------------+-----+
only showing top 1 row

Department: Animal Care Services
+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|26760|
+--------------------+-----+
only showing top 1 row

Department: Trans & Cap Improvements
+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|     Pot Hole Repair|20616|
+--------------------+-----+
only showing top 1 row

Department: Parks & Recreation
+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           Carpentry| 1944|
+--------------------+-----+
only showing top 1 row

Department: Customer Service
+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           Complaint| 2388|
+--------------------+-----+
only showing top 1 row

D